In [1]:
import ee
import geemap
ee.Initialize()

In [2]:
Map = geemap.Map()

In [3]:
# 研究区
boundry = 'projects/ee-chenximing2021/assets/jiefangzha_shape'

polygon = ee.FeatureCollection(boundry).geometry()
Map.addLayer(polygon,{},'polygon')
# Map

In [4]:
# 添加毫秒计数的时间属性
def addTimeProp(image):
    date = image.getInfo()['properties']['date']
    image = image.set('system:time_start',ee.Date(date).millis()) 
    return image

In [5]:
# 添加时间波段
def addTimeBand(image):
    timeImage = image.metadata('system:time_start').rename('timestamp')
    timeImageMasked = timeImage.updateMask(image.mask().select(0))
    return image.addBands(timeImageMasked)

In [6]:
# 导入去云和大气校正后的影像

folder = r'projects/ee-chenximing2021/assets/'
nums = 87
S2 = []

for num in range(nums):
    id = folder + '2019_' + str(num)
    image = ee.Image(id)
    # 添加时间属性
    image = addTimeProp(image)
    # 添加时间波段
    image = addTimeBand(image)
    S2.append(image)

print('{} image loaded successgfully from the GEE Assests'.format(len(S2)))
   
S2 = ee.ImageCollection.fromImages(S2)

87 image loaded successgfully from the GEE Assests


In [7]:
# Map.addLayer(S2.first(),{'bands':['B4','B3','B2'],'min':0,'max':0.5},'s2 cloud masked and atmocor image')
# Map

插值

In [8]:
# 设置插值的时间窗口，筛选条件
days = 60
millis = ee.Number(days).multiply(1000*60*60*24)

#过滤器，筛选指定时间范围内的影像
maxDiffFilter = ee.Filter.maxDifference(**{
    'difference': millis,
    'leftField': 'system:time_start',
    'rightField': 'system:time_start',
})
# 过滤器，筛选出一个影像之后（日期）的影像
lessEqFilter = ee.Filter.lessThanOrEquals(**{
    'leftField': 'system:time_start',
    'rightField': 'system:time_start'
})
# 过滤器，筛选出一个影像之前（日期）的影像
greaterEqFilter = ee.Filter.greaterThanOrEquals(**{
    'leftField': 'system:time_start',
    'rightField': 'system:time_start'
})
# 结合两个过滤器，筛选出在日期范围内的影像，这里 image collection 与自身进行匹配，并过滤
# 在给定时间跨度内，筛选出 image collection 中每景影像开始时间后的影像
filter1 = ee.Filter.And(maxDiffFilter, lessEqFilter)
# 将一个iamge collection 中的每个影像分别与另一个 image collection 的一组影像（时间范围days筛选出的影像）进行匹配，筛选出符合规则的影像
# join1 将符合规则的影像以列表的形式添加到影像中的属性中，属性名称为 matchsKey 所指示
join1 = ee.Join.saveAll(**{
    'matchesKey': 'after',  # 用于保存符合规则的影像的列表的属性名
    'ordering': 'system:time_start',    # 用于筛选符合匹配规则的影像的属性
    'ascending': False})
# 应用 匹配规则 进行筛选
join1Result = join1.apply(**{
    'primary': S2,  
    'secondary': S2,
    'condition': filter1
})

# 下面的筛选同上
filter2 = ee.Filter.And(maxDiffFilter, greaterEqFilter)
 
join2 = ee.Join.saveAll(**{
    'matchesKey': 'before',
    'ordering': 'system:time_start',
    'ascending': True})
   
join2Result = join2.apply(**{
    'primary': join1Result,
    'secondary': join1Result,
    'condition': filter2
})


In [9]:
def interpolateImages(image):
    image = ee.Image(image)
    beforeImages = ee.List(image.get('before'))
    beforeMosaic = ee.ImageCollection.fromImages(beforeImages).mosaic()
    afterImages = ee.List(image.get('after'))
    afterMosaic = ee.ImageCollection.fromImages(afterImages).mosaic()
   
    t1 = beforeMosaic.select('timestamp').rename('t1')
    t2 = afterMosaic.select('timestamp').rename('t2')
    t = image.metadata('system:time_start').rename('t')
    timeImage = ee.Image.cat([t1,t2,t])
    timeRatio = timeImage.expression('(t-t1)/(t2-t1)', {'t':timeImage.select('t'), 't1':timeImage.select('t1'), 't2':timeImage.select('t2')})

    interpolated = beforeMosaic.add((afterMosaic.subtract(beforeMosaic).multiply(timeRatio)))
    result = image.unmask(interpolated)
    return result.copyProperties(image, ['system:time_start'])

In [10]:
# 对数据集进行插值
interpolatedCol = ee.ImageCollection(join2Result.map(interpolateImages))

In [11]:
interList = interpolatedCol.toList(interpolatedCol.size())
s2List = S2.toList(S2.size())

In [31]:
inte = ee.Image(interList.get(5))
origin = ee.Image(s2List.get(5))

In [32]:
Map.addLayer(origin ,{'bands':['B4','B3','B2'],'min':0,'max':0.5},'origin')
Map.addLayer(inte ,{'bands':['B4','B3','B2'],'min':0,'max':0.5},'interpolated')
Map

Map(bottom=98566.0, center=[41.01928287604562, 107.19635009765626], controls=(WidgetControl(options=['position…

S-G滤波

In [14]:
bands = ['B2', 'B3', 'B4', 'B5', 'B6', 'B7', 'B8A', 'B11', 'B12']

In [15]:
start_date = '2019-01-01'
smoothedCol_bands = []
for index, band in enumerate(bands):
    def addBands(image):
        dstamp = ee.Date(image.get('system:time_start'))
        diff = dstamp.difference(ee.Date(start_date),'day')
        image = image.select(band).set('date',dstamp).clip(polygon).set('footprint',polygon)
        return image.addBands(ee.Image(1).toFloat().rename('constant')).addBands(ee.Image(diff).toFloat().rename('t')).addBands(ee.Image(diff).pow(ee.Image(2)).toFloat().rename('t2')).addBands(ee.Image(diff).pow(ee.Image(3)).toFloat().rename('t3'))    

    interpolatedCol_res = interpolatedCol.map(addBands)
    
    window_size = 9
    half_window = (window_size - 1)/2
    imageAxis = 0
    bandAxis = 1

    order = 3
    coeffFlattener = [['constant', 'x', 'x2', 'x3']]
    indepSelectors = ['constant', 't', 't2', 't3']

    array = interpolatedCol_res.toArray()
    interpolatedCol_res = interpolatedCol_res.toList(interpolatedCol_res.size())
    runLength = ee.List.sequence(0, interpolatedCol_res.size().subtract(window_size))


    def getLocalFit(i):
        subarray = array.arraySlice(imageAxis, ee.Number(i).int(), ee.Number(i).add(window_size).int())
        predictors = subarray.arraySlice(bandAxis, 1, 1+order+1)    
        response = subarray.arraySlice(bandAxis, 0, 1)
        coeff = predictors.matrixSolve(response)
        coeff = coeff.arrayProject([0]).arrayFlatten(coeffFlattener)
        return coeff


    def smooth(i):
        ref = ee.Image(interpolatedCol_res.get(ee.Number(i).add(half_window)))
        return getLocalFit(i).multiply(ref.select(indepSelectors)).reduce(ee.Reducer.sum()).copyProperties(ref)
    
    sg_series = runLength.map(smooth)
    # sg_series = ee.ImageCollection(runLength.map(smooth))
    print(type(sg_series))

    # def rename(image):
    #     return image.select('sum').rename(band+"_sum")
    # sg_series = sg_series.map(rename)
    # Map.addLayer(ee.Image(sg_series.first()),{'bands':[band+'_sum'],'min':0,'max':0.5},'smooth_2_' + band)
    
    smoothedCol_bands.append(sg_series)



<class 'ee.ee_list.List'>
<class 'ee.ee_list.List'>
<class 'ee.ee_list.List'>
<class 'ee.ee_list.List'>
<class 'ee.ee_list.List'>
<class 'ee.ee_list.List'>
<class 'ee.ee_list.List'>
<class 'ee.ee_list.List'>
<class 'ee.ee_list.List'>


In [16]:
# Map

In [17]:
smoothedcol_B2 = smoothedCol_bands[0]
smoothedCol_B3 = smoothedCol_bands[1]
smoothedCol_B4 = smoothedCol_bands[2]
smoothedCol_B5 = smoothedCol_bands[3]
smoothedCol_B6 = smoothedCol_bands[4]
smoothedCol_B7 = smoothedCol_bands[5]
smoothedCol_B8A = smoothedCol_bands[6]
smoothedCol_B11 = smoothedCol_bands[7]
smoothedCol_B12 = smoothedCol_bands[8]

# b2 =  ee.Image(smoothedcol_B2.get(0))
# Map.addLayer(b2,{'bands':['B2'],'min':0,'max':0.5}, 'b2')

def mergeBand(i):
    index = smoothedcol_B2.indexOf(i)
    return ee.Image(i).select('sum').rename('B2_sum').addBands(ee.Image(smoothedCol_B3.get(index)).rename('B3_sum')).addBands(ee.Image(smoothedCol_B4.get(index)).rename('B4_sum'))\
    .addBands(ee.Image(smoothedCol_B5.get(index)).rename('B5_sum')).addBands(ee.Image(smoothedCol_B6.get(index)).rename('B6_sum')).addBands(ee.Image(smoothedCol_B7.get(index)).rename('B7_sum'))\
    .addBands(ee.Image(smoothedCol_B8A.get(index)).rename('B8A_sum')).addBands(ee.Image(smoothedCol_B11.get(index)).rename('B11_sum')).addBands(ee.Image(smoothedCol_B12.get(index)).rename('B12_sum'))

s2SmoothedCol = smoothedcol_B2.map(mergeBand)
s2SmoothedCol = ee.ImageCollection(s2SmoothedCol)
img1 = s2SmoothedCol.first()
Map.addLayer(img1,{'bands':['B4_sum','B3_sum','B2_sum'],'min':0,'max':0.5}, 'img1')



In [33]:
print(inte.getInfo()['properties'])

{'system:time_start': 1549411200000, 'before': [{'type': 'Image', 'bands': [{'id': 'B2', 'data_type': {'type': 'PixelType', 'precision': 'double'}, 'dimensions': [7987, 8916], 'crs': 'EPSG:32648', 'crs_transform': [10, 0, 630740, 0, -10, 4574470]}, {'id': 'B3', 'data_type': {'type': 'PixelType', 'precision': 'double'}, 'dimensions': [7987, 8916], 'crs': 'EPSG:32648', 'crs_transform': [10, 0, 630740, 0, -10, 4574470]}, {'id': 'B4', 'data_type': {'type': 'PixelType', 'precision': 'double'}, 'dimensions': [7987, 8916], 'crs': 'EPSG:32648', 'crs_transform': [10, 0, 630740, 0, -10, 4574470]}, {'id': 'B5', 'data_type': {'type': 'PixelType', 'precision': 'double'}, 'dimensions': [7987, 8916], 'crs': 'EPSG:32648', 'crs_transform': [10, 0, 630740, 0, -10, 4574470]}, {'id': 'B6', 'data_type': {'type': 'PixelType', 'precision': 'double'}, 'dimensions': [7987, 8916], 'crs': 'EPSG:32648', 'crs_transform': [10, 0, 630740, 0, -10, 4574470]}, {'id': 'B7', 'data_type': {'type': 'PixelType', 'precision'

In [56]:
smoothed_list = s2SmoothedCol.toList(s2SmoothedCol.size())
smoothed = ee.Image(smoothed_list.get(1))

In [58]:
Map.addLayer(smoothed,{'bands':['B4_sum','B3_sum','B2_sum'],'min':0,'max':0.5}, 'smoothed')

In [59]:
Map

Map(bottom=98566.0, center=[41.01928287604562, 107.19635009765626], controls=(WidgetControl(options=['position…